In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import math
from matplotlib import pyplot as plt

In [2]:
from nuplan_extent.planning.training.modeling.objectives.repulsive_interaction_objective import RepulsiveInteractionObjective

In [3]:
B = 5
N1 = 11
M = 6
Tf = 16

In [17]:
predictions = {}
for i in range(0, 7):
    predictions[f"trajectories_{i}"] = torch.randn(B,N1,M,Tf,4, requires_grad=True)

In [18]:
loss = RepulsiveInteractionObjective(
    safety_margin=0.5,
    k_level=6
)

In [19]:
output = loss.compute(predictions, None, None)

In [21]:
output.backward()

Fake a pseudo prediction.

2 agents, 2 modalities, 16 steps

In [7]:
a1m1 = torch.stack([torch.arange(16), torch.zeros(16)], dim=-1)
a1m2 = torch.stack([torch.arange(16), torch.arange(16)*torch.tan(torch.tensor(math.pi/20))], dim=-1)
a2m1 = torch.stack([torch.arange(16), -torch.arange(16)*torch.tan(torch.tensor(math.pi/40))], dim=-1)
a2m2 = torch.stack([torch.arange(16), -torch.arange(16)*torch.tan(torch.tensor(math.pi/60))], dim=-1)

In [8]:
a1 = torch.stack([a1m1, a1m2], dim=0)
a2 = torch.stack([a2m1, a2m2], dim=0)

In [9]:
a = torch.stack([a1, a2], dim=0).unsqueeze(0)

In [10]:
a.shape # B=1, N1=2, M=2, Tf=16, 2

torch.Size([1, 2, 2, 16, 2])

In [11]:
loss = RepulsiveInteractionObjective(
    safety_margin=0.5,
    k_level=1
)

loss.compute({"trajectories_0": a, "trajectories_1": a}, None, None)

tensor(6.4000)

In [22]:
input_lane_dim=12
hidden_feature_dim=256
map_lane_encoder = nn.Sequential(
            nn.Linear(input_lane_dim, hidden_feature_dim),
            nn.ReLU6(),
            nn.Linear(hidden_feature_dim, hidden_feature_dim),
            nn.ReLU6(),
        )

In [25]:
list(map_lane_encoder.named_parameters())

[('0.weight',
  Parameter containing:
  tensor([[ 0.2413,  0.0747, -0.2303,  ..., -0.2195,  0.1393,  0.0543],
          [-0.1195, -0.0750, -0.2877,  ..., -0.1785,  0.2412,  0.1930],
          [-0.2373,  0.2789,  0.2779,  ..., -0.0102,  0.1358, -0.1909],
          ...,
          [ 0.0652,  0.2340,  0.0939,  ..., -0.2657,  0.2819,  0.2617],
          [-0.2064,  0.0654, -0.2812,  ...,  0.2061,  0.2790, -0.1653],
          [-0.1287, -0.1573, -0.0054,  ..., -0.2163,  0.0419,  0.0042]],
         requires_grad=True)),
 ('0.bias',
  Parameter containing:
  tensor([-2.3387e-01,  2.0751e-01,  2.8650e-01,  2.5986e-01, -2.0627e-01,
           7.2616e-02,  1.4986e-01,  3.2320e-02, -7.0510e-02,  4.6771e-02,
           2.8569e-01,  2.5051e-02, -1.7815e-01, -2.7981e-01, -2.1680e-01,
          -4.0761e-02, -1.2585e-01,  5.2066e-02, -7.9540e-03, -2.1127e-01,
           4.4438e-02,  1.8010e-01,  2.7489e-01,  2.4579e-01, -6.8049e-02,
           6.4941e-03,  2.8817e-01, -8.2870e-02,  1.8270e-01, -1.2428e-0